In [ ]:
import keras

import numpy as np
import pandas as pd
import seaborn as sn
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import matplotlib.dates as mds

def encode(series):
    return pd.get_dummies(series.astype(str))

In [ ]:
model = keras.models.load_model('cnnr_iter1.h5')
model.summary()
pred_labels_all = []

In [ ]:
#FNN CLASSIFICATION

features = np.loadtxt('vec_mat_features_icex_src_icex16_test.csv',delimiter=",")
temp_labels = np.loadtxt('vec_mat_clabels_icex_src_test2.csv',delimiter=",")
labels_t = []

X_test = preprocessing.scale(features)

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))
    
labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series): 
    return pd.get_dummies(series.astype(str))

y_test = encode(labels_t)
labels = list(y_test.columns.values)

print(X_test.shape)

predictions = model.predict(X_test)

pred_labels = []
for i in np.argmax(predictions, axis=1):
    pred_labels.append(float(labels[i]))
    
time = np.arange(0,predictions.shape[0]*10.2412,10.2412)

In [ ]:
floatcn = np.array(labels).astype(np.float)
inds = floatcn.argsort()
sorted_predictions = predictions[:,inds]

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

c = ax.pcolormesh(time, np.linspace(3,50,95), sorted_predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.scatter(time,pred_labels,c='r',marker='.')
ax.set_title('ICEX16 Hour 8 FNN Classification Result (With Normalization)',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
ax.set_xlabel('Time (s)',fontsize=20)
ax.axis([0,time[-1], 3, 50],fontsize=20)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('fnn_class_norm_icex8.png')

In [ ]:
# FNN REGRESSION TESTING
#Import testing data
X_test = np.loadtxt('vec_mat_features_icex_src_icex16_hour8_norm.csv',delimiter=",")

X_test = preprocessing.scale(X_test)

print(X_test.shape)

predictions = model.predict(X_test)


In [ ]:
#Plotting
fig1 = plt.figure(figsize = (10,8))
plt.scatter(time,np.squeeze(predictions),marker='.',c='r',alpha=0.75)
plt.axis([0,time[-1], 3, 50],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Time (s)',fontsize=20)
plt.title('ICEX16 Hour 8 FNN Regression Test Predictions (With Normalization)',fontsize=20)
plt.grid()

In [ ]:
fig1.savefig('fnn_reg_norm_icex8.png')

In [ ]:
#CNN Classification testing
features_test = np.loadtxt('vec_mat_features_icex_src_icex16_hour8_norm.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_clabels_icex_src_test2_norm.csv',delimiter=",")
y_test= []

real_test = features_test[:,0::2]
imag_test = features_test[:,1::2]
X_test = np.zeros((features_test.shape[0],32,32,2))

for k in range(features_test.shape[0]):
    count = 0
    for i in range(32):
        for j in range(i,32):
            X_test[k,i,j,0] = real_test[k,count]
            X_test[k,i,j,1] = imag_test[k,count]
            
            if i!=j:
                X_test[k,j,i,0] = X_test[k,i,j,0]
                X_test[k,j,i,1] = -X_test[k,i,j,1]
            
            count = count + 1

for l in range(len(temp_ytest)):
    y_test.append(str(temp_ytest[l]))

y_test = np.array(y_test)
y_test = y_test.ravel()
label_test = y_test.ravel()

y_test = encode(y_test)
class_names = list(y_test.columns.values)


predictions = model.predict(X_test)

pred_labels = []
for i in np.argmax(predictions, axis=1):
    pred_labels.append(float(class_names[i]))
    



In [ ]:
pred_labels_all = pred_labels_all+pred_labels 

In [ ]:
ep_startt = 1457852733 #38m
time = np.arange(0,len(pred_labels_all)*10.92,10.92) + ep_startt

In [ ]:
time = np.arange(0,len(pred_labels_all)*10.92,10.92)

In [ ]:
floatcn = np.array(class_names).astype(np.float)
inds = floatcn.argsort()
sorted_predictions = predictions[:,inds]

In [ ]:
window_size = 55

i = 0
moving_averages = []
while i < len(pred_labels_all) - window_size + 1:
    this_window = pred_labels_all[i : i + window_size]
    window_average = np.sum(this_window) / window_size
    moving_averages.append(window_average)
    i += 1

In [ ]:
len(moving_averages)

In [ ]:

fig, ax = plt.subplots(figsize=(20,8))

#c = ax.pcolormesh(time, np.linspace(3,50,95), sorted_predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.scatter(mds.epoch2num(time),pred_labels_all,c='b',marker='.')
ax.plot(mds.epoch2num(time[int(window_size/2):int(-(window_size/2))]),moving_averages,'r')
ax.set_title('ICEX16 38m-depth CNN-c',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
plt.xlabel('Time (Mar. 13, 2016 UTC)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.ylim(3,50)
ax1 = plt.gca()
ax1.xaxis.set_major_formatter(formatter=mds.DateFormatter('%H:%M:%S'))
ax.axis([mds.epoch2num(time[0]),mds.epoch2num(time[-1]), 3, 50],fontsize=15)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
#fig.colorbar(c, ax=ax)
plt.grid()
plt.legend(('10-min moving avg.','sample predictions'),fontsize=20,loc='lower right')
plt.tight_layout()
#fig.autofmt_xdate()
plt.savefig('icex_38m_cnnc.png')
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(20,8))

#c = ax.pcolormesh(time, np.linspace(3,50,95), sorted_predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.scatter(time,pred_labels_all,c='b',marker='.')
ax.plot(time[int(window_size/2):int(-(window_size/2))],moving_averages,'r')
ax.set_title('ICEX16 138m-depth CNN-c',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
plt.xlabel('Time (sec)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.ylim(3,50)
ax1 = plt.gca()
#ax1.xaxis.set_major_formatter(formatter=mds.DateFormatter('%H:%M:%S'))
ax.axis([time[0],time[-1], 3, 50],fontsize=15)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
#fig.colorbar(c, ax=ax)
plt.grid()
plt.legend(('10-min moving avg.','sample predictions'),fontsize=20)
plt.tight_layout()
#fig.autofmt_xdate()
plt.savefig('icex_138m_cnnc.png')
plt.show()

In [ ]:
fig.savefig('cnn5l_class_norm_icex138m.png')

In [ ]:
#CNN Regression Testing

features_test = np.loadtxt('vec_mat_features_icex_src_icex16_hour8_norm.csv',delimiter=",")

real_test = features_test[:,0::2]
imag_test = features_test[:,1::2]
X_test = np.zeros((features_test.shape[0],32,32,2))

for k in range(features_test.shape[0]):
    count = 0
    for i in range(32):
        for j in range(i,32):
            X_test[k,i,j,0] = real_test[k,count]
            X_test[k,i,j,1] = imag_test[k,count]
            
            if i!=j:
                X_test[k,j,i,0] = X_test[k,i,j,0]
                X_test[k,j,i,1] = -X_test[k,i,j,1]
            
            count = count + 1


predictions = model.predict(X_test)
time = np.arange(0,predictions.shape[0]*10.2412,10.2412)

In [ ]:
pred_labels_all = pred_labels_all+predictions.tolist() 

In [ ]:
print(time.shape)
print(len(pred_labels_all))

In [ ]:

fig, ax = plt.subplots(figsize=(20,8))

#c = ax.pcolormesh(time, np.linspace(3,50,95), sorted_predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.scatter(mds.epoch2num(time),pred_labels_all,c='b',marker='.')
ax.plot(mds.epoch2num(time[int(window_size/2):int(-(window_size/2))]),moving_averages,'r')
ax.set_title('ICEX16 38m-depth CNN-r',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
plt.xlabel('Time (Mar. 13, 2016 UTC)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.ylim(3,50)
ax1 = plt.gca()
ax1.xaxis.set_major_formatter(formatter=mds.DateFormatter('%H:%M:%S'))
ax.axis([mds.epoch2num(time[0]),mds.epoch2num(time[-1]), 3, 50],fontsize=15)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
#fig.colorbar(c, ax=ax)
plt.grid()
plt.legend(('10-min moving avg.','sample predictions'),fontsize=20,loc='lower right')
plt.tight_layout()
#fig.autofmt_xdate()
plt.savefig('icex_38m_cnnr.png')
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(20,8))

#c = ax.pcolormesh(time, np.linspace(3,50,95), sorted_predictions.T*100, cmap='jet',vmin=0, vmax=100)
ax.scatter(time,pred_labels_all,c='b',marker='.')
ax.plot(time[int(window_size/2):int(-(window_size/2))],moving_averages,'r')
ax.set_title('ICEX16 238m-depth CNN-r',fontsize=20)
ax.set_ylabel('Predicted Source Distance (km)',fontsize=20)
plt.xlabel('Time (sec)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.ylim(3,50)
ax1 = plt.gca()
#ax1.xaxis.set_major_formatter(formatter=mds.DateFormatter('%H:%M:%S'))
ax.axis([time[0],time[-1], 3, 50],fontsize=15)
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
#fig.colorbar(c, ax=ax)
plt.grid()
plt.legend(('10-min moving avg.','sample predictions'),fontsize=20)
plt.tight_layout()
#fig.autofmt_xdate()
plt.savefig('icex_238m_cnnr.png')
plt.show()

In [ ]:

#Plotting
fig1 = plt.figure(figsize = (10,8))
plt.scatter(time,np.squeeze(predictions),marker='.',c='r',alpha=0.75)
plt.axis([0,time[-1], 3, 50],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Time (s)',fontsize=20)
plt.title('ICEX16 138m CNN Regression Test Predictions (With Normalization)',fontsize=20)
plt.grid()

In [ ]:
fig1.savefig('cnn5l_reg_norm_icex138m.png')